In [1]:
import torch
import torchvision 
import torch.nn.functional as F  
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms
from torch import optim  # For optimizers SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader
from tqdm import tqdm # For nice progress bar!

build a simple convolutional neural network

In [2]:
class CNN(nn.Module):
    def __init__(self , in_channels = 1 , num_classes = 10 ):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels = in_channels,
            out_channels = 8 ,
            kernel_size = (3,3),
            stride = (1,1),
            padding = (1,1)
        )
        
        self.pool = nn.MaxPool2d(
            kernel_size=(2, 2), 
            stride=(2, 2)
        )
        
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.fc = nn.Linear(16 * 7 * 7, num_classes)
    def forward(self , x ):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)       
        ### reshape in forward process
        x = x.reshape(x.shape[0], -1)
        
        x = self.fc(x)
        return x

# Save and Load model

In [3]:
def save_checkpoint(state , filename = 'my_checkpoint.pth'): # default name
    print("===> Saving checkpoint")
    torch.save(state , filename)

def load_checkpoint(checkpoint, model , optimizer): # default name
    print("===> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    print(checkpoint["epoch"])

set device

In [4]:
target_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameter

In [5]:
in_channels = 1
input_size = 784
num_classes = 10
learning_rate = 1e-3
batch_size = 64
num_epochs = 3



Load Training and Test data. Usually need to write dataloaber ourselves.

In [6]:

test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=False)
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=False)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
# Initialize network

In [8]:
model = CNN(in_channels=in_channels , num_classes=num_classes).to(target_device)

In [9]:
# Loss and optimizer

In [10]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train the Network

In [11]:
##########################################
load_model = True
##########################################
if load_model == True:
    load_checkpoint(torch.load("my_checkpoint.pth"), model, optimizer)

In [12]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=target_device)
            y = y.to(device=target_device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples




In [13]:
for epoch in range(num_epochs):
    if(epoch % 1 == 0):
        ### checkpoint is a dictionary don't forget () after dict()!!!!!!!!!!!!!!!!!!!!!
        ### using .state_dict() is recommended
        checkpoint = {"state_dict":model.state_dict() , "optimizer":optimizer.state_dict() , 'epoch':epoch}

        save_checkpoint(checkpoint)
        
    for batch_idx , ( data, targets ) in enumerate( tqdm( train_loader ) ):
        # Get data to cuda if possible
        data = data.to( device = target_device )
        targets = targets.to( device = target_device )

        # Get to correct shape
        # data.shape[0] usually is the number of examples in a batch
        # in this case, just set each sample to a long bar
        # data = data.reshape(data.shape[0], -1) this is done before fully connected layer
        # not in CNN

        # forward
        predictions = model(data)
        #targets are true labels
        loss = loss_func(predictions , targets)

        # backward
        optimizer.zero_grad() # set the gradients to zero, so that we are ready for the next loop
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        
        # usually we don't do this, here just for illustration
#print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
#print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

  0%|                                                                                          | 0/938 [00:00<?, ?it/s]

===> Saving checkpoint


  1%|█                                                                               | 13/938 [00:00<00:08, 115.23it/s]

===> Saving checkpoint


  1%|█                                                                               | 12/938 [00:00<00:07, 119.60it/s]

===> Saving checkpoint


100%|███████████████████████████████████████████████████████████████████████████████| 938/938 [00:07<00:00, 119.80it/s]
